In [ ]:
# Cell 1: Install Required Packages (WITH GROQ)
!pip install -q pyngrok streamlit joblib scikit-learn pandas openpyxl groq
!pip install -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 39.8 MB/s eta 0:00:00


In [ ]:
# Cell 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Cell 3: Copy Files from Drive
import os
import shutil

# Create models directory
!mkdir -p /content/models_adaptive

# Copy model files
!cp "/content/drive/MyDrive/SMH/models_adaptive/"* /content/models_adaptive/ 2>/dev/null || echo "Model files copy failed"

# Copy data files
!cp "/content/drive/MyDrive/SMH/X_cleaned.xls" /content/
!cp "/content/drive/MyDrive/SMH/y_cleaned.xls" /content/

# Copy JSON files
!cp "/content/drive/MyDrive/SMH/disease_info (1).json" "/content/disease_info.json"
!cp "/content/drive/MyDrive/SMH/features.json" /content/

# Verify files
print("\n✅ Files in /content/models_adaptive:")
!ls -lh /content/models_adaptive/

print("\n✅ Files in /content:")
!ls -lh /content/*.json /content/*.xls 2>/dev/null || echo "Some files missing"

Mounted at /content/drive

✅ Files in /content/models_adaptive:
total 3.1G
-rw------- 1 root root  16M Dec 11 14:31  DecisionTree.pkl
-rw------- 1 root root 517K Dec 11 14:31 'disease_info (1).json'
-rw------- 1 root root 7.7K Dec 11 14:31  features.json
-rw------- 1 root root 153M Dec 11 14:31  KNN.pkl
-rw------- 1 root root 617K Dec 11 14:31  LogisticRegression.pkl
-rw------- 1 root root 301K Dec 11 14:31  pca.pkl
-rw------- 1 root root 2.9G Dec 11 14:32  RandomForest.pkl
-rw------- 1 root root  19K Dec 11 14:32  scaler.pkl

✅ Files in /content:
-rw------- 1 root root 517K Dec 11 14:32 /content/disease_info.json
-rw------- 1 root root 7.7K Dec 11 14:32 /content/features.json
-rw------- 1 root root 356M Dec 11 14:32 /content/X_cleaned.xls
-rw------- 1 root root 4.7M Dec 11 14:32 /content/y_cleaned.xls


In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import joblib
import json
import os
from datetime import datetime

# Try to import Groq
try:
    from groq import Groq
    GROQ_AVAILABLE = True
except ImportError:
    GROQ_AVAILABLE = False

# ---------------- CONFIG ----------------
MODEL_DIR = "models_adaptive"
DISEASE_INFO_PATH = "disease_info.json"
FEATURE_FILE = "features.json"

# 🔑 HARDCODED GROQ API KEY (Replace with your key)
GROQ_API_KEY = "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"

# 👨‍⚕️ SPECIALIST DOCTORS DATABASE
DOCTORS_DATABASE = {
    "general": [
        {"name": "Dr. Rajesh Kumar", "specialty": "General Physician", "experience": "15 years", "hospital": "Apollo Hospital", "phone": "+91-9876543210", "available": "Mon-Sat, 9AM-5PM"},
        {"name": "Dr. Priya Sharma", "specialty": "General Physician", "experience": "12 years", "hospital": "Fortis Healthcare", "phone": "+91-9876543211", "available": "Mon-Fri, 10AM-6PM"},
    ],
    "cardiology": [
        {"name": "Dr. Amit Patel", "specialty": "Cardiologist", "experience": "20 years", "hospital": "Max Hospital", "phone": "+91-9876543212", "available": "Mon-Sat, 8AM-4PM"},
        {"name": "Dr. Sneha Reddy", "specialty": "Cardiologist", "experience": "18 years", "hospital": "Apollo Hospital", "phone": "+91-9876543213", "available": "Tue-Sun, 9AM-5PM"},
    ],
    "dermatology": [
        {"name": "Dr. Kavita Iyer", "specialty": "Dermatologist", "experience": "10 years", "hospital": "Skin Care Clinic", "phone": "+91-9876543214", "available": "Mon-Sat, 10AM-7PM"},
        {"name": "Dr. Arjun Mehta", "specialty": "Dermatologist", "experience": "14 years", "hospital": "Fortis Healthcare", "phone": "+91-9876543215", "available": "Mon-Fri, 11AM-6PM"},
    ],
    "neurology": [
        {"name": "Dr. Suresh Menon", "specialty": "Neurologist", "experience": "22 years", "hospital": "Neuro Care Center", "phone": "+91-9876543216", "available": "Mon-Sat, 9AM-4PM"},
        {"name": "Dr. Lakshmi Nair", "specialty": "Neurologist", "experience": "16 years", "hospital": "Max Hospital", "phone": "+91-9876543217", "available": "Tue-Sat, 10AM-5PM"},
    ],
    "orthopedic": [
        {"name": "Dr. Vikram Singh", "specialty": "Orthopedic Surgeon", "experience": "19 years", "hospital": "Bone & Joint Clinic", "phone": "+91-9876543218", "available": "Mon-Fri, 8AM-3PM"},
        {"name": "Dr. Meera Joshi", "specialty": "Orthopedic Surgeon", "experience": "13 years", "hospital": "Apollo Hospital", "phone": "+91-9876543219", "available": "Mon-Sat, 9AM-6PM"},
    ],
    "gastroenterology": [
        {"name": "Dr. Rahul Verma", "specialty": "Gastroenterologist", "experience": "17 years", "hospital": "Digestive Care Hospital", "phone": "+91-9876543220", "available": "Mon-Sat, 10AM-5PM"},
        {"name": "Dr. Anjali Kapoor", "specialty": "Gastroenterologist", "experience": "11 years", "hospital": "Fortis Healthcare", "phone": "+91-9876543221", "available": "Tue-Sun, 9AM-4PM"},
    ],
    "pediatrics": [
        {"name": "Dr. Rohit Desai", "specialty": "Pediatrician", "experience": "14 years", "hospital": "Children's Hospital", "phone": "+91-9876543222", "available": "Mon-Sat, 8AM-6PM"},
        {"name": "Dr. Pooja Gupta", "specialty": "Pediatrician", "experience": "10 years", "hospital": "Max Hospital", "phone": "+91-9876543223", "available": "Mon-Fri, 9AM-7PM"},
    ],
    "ent": [
        {"name": "Dr. Karthik Rao", "specialty": "ENT Specialist", "experience": "16 years", "hospital": "ENT Care Center", "phone": "+91-9876543224", "available": "Mon-Sat, 10AM-6PM"},
        {"name": "Dr. Divya Pillai", "specialty": "ENT Specialist", "experience": "12 years", "hospital": "Apollo Hospital", "phone": "+91-9876543225", "available": "Tue-Sat, 9AM-5PM"},
    ],
}

# Disease to specialty mapping
DISEASE_SPECIALTY_MAP = {
    "diabetes": "general",
    "hypertension": "cardiology",
    "heart disease": "cardiology",
    "skin disease": "dermatology",
    "acne": "dermatology",
    "migraine": "neurology",
    "epilepsy": "neurology",
    "arthritis": "orthopedic",
    "fracture": "orthopedic",
    "gastritis": "gastroenterology",
    "ulcer": "gastroenterology",
    "fever": "general",
    "cold": "ent",
    "asthma": "general",
    "pneumonia": "general",
}

# ---------------- GROQ CHATBOT ----------------
def get_health_response(user_query, chat_history):
    """Get health response from Groq"""

    if not GROQ_AVAILABLE:
        return "❌ Groq library not installed."

    if not GROQ_API_KEY or GROQ_API_KEY == "gsk_YOUR_API_KEY_HERE":
        return "❌ Please add your Groq API key in the code (line 18)."

    try:
        client = Groq(api_key=GROQ_API_KEY)

        system_prompt = """You are a compassionate health assistant.

Answer ONLY health and medical questions with:
- Clear, simple explanations
- Empathetic tone
- Practical advice

For non-health questions, strictly inform that you are a health bot.

Always end with: "⚠️ I'm an AI assistant. Consult healthcare professionals for medical advice."

Be warm and helpful."""

        messages = [{"role": "system", "content": system_prompt}]

        # Add recent history
        recent_history = chat_history[-4:] if len(chat_history) > 4 else chat_history
        for msg in recent_history:
            if msg["role"] in ["user", "assistant"]:
                messages.append({
                    "role": msg["role"],
                    "content": msg["content"]
                })

        messages.append({"role": "user", "content": user_query})

        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=messages,
            temperature=0.5,
            max_tokens=600,
            timeout=30.0
        )

        return response.choices[0].message.content

    except Exception as e:
        return f"❌ Error: {str(e)[:200]}"

# ---------------- LOAD MODELS ----------------
@st.cache_resource
def load_resources():
    try:
        scaler_path = os.path.join(MODEL_DIR, "scaler.pkl")
        pca_path = os.path.join(MODEL_DIR, "pca.pkl")

        if not os.path.exists(scaler_path) or not os.path.exists(pca_path):
            return None, None, None, None, None

        scaler = joblib.load(scaler_path)
        pca = joblib.load(pca_path)

        disease_info = {}
        if os.path.exists(DISEASE_INFO_PATH):
            with open(DISEASE_INFO_PATH, "r") as f:
                disease_info = json.load(f)

        features = []
        if os.path.exists(FEATURE_FILE):
            with open(FEATURE_FILE, "r") as f:
                features = json.load(f)

        models = {}
        for model_file in os.listdir(MODEL_DIR):
            if model_file.endswith(".pkl") and model_file not in ["scaler.pkl", "pca.pkl"]:
                model_name = model_file.replace(".pkl", "")
                model_path = os.path.join(MODEL_DIR, model_file)
                try:
                    models[model_name] = joblib.load(model_path)
                except:
                    pass

        return disease_info, features, models, scaler, pca

    except Exception as e:
        st.error(f"❌ Error loading: {e}")
        return None, None, None, None, None

# ---------------- PREDICTION ----------------
def calculate_match_percentage(input_symptoms, disease_symptoms):
    if not disease_symptoms:
        return 0
    matches = len(set(input_symptoms) & set([s.lower() for s in disease_symptoms]))
    return round((matches / len(disease_symptoms)) * 100, 2)

def predict_disease(symptoms, disease_info, features, models, scaler, pca):
    input_vector = np.zeros(len(features))

    for sym in symptoms:
        sym = sym.strip().lower()
        if sym in features:
            input_vector[features.index(sym)] = 1

    if sum(input_vector) == 0:
        return []

    X_scaled = scaler.transform([input_vector])
    X_pca = pca.transform(X_scaled)

    results = []
    for model_name, model in models.items():
        try:
            pred = model.predict(X_pca)[0]
            info = disease_info.get(pred.lower(), {})
            disease_symptoms = info.get("symptoms", [])
            match_percent = calculate_match_percentage(symptoms, disease_symptoms)
            results.append((model_name, pred, match_percent, info))
        except:
            pass

    results.sort(key=lambda x: x[2], reverse=True)
    return results

# ---------------- DOCTOR FINDER ----------------
def get_specialists_for_disease(disease_name):
    """Get relevant specialists based on disease"""
    disease_lower = disease_name.lower()

    # Find matching specialty
    specialty = "general"
    for disease_key, spec in DISEASE_SPECIALTY_MAP.items():
        if disease_key in disease_lower:
            specialty = spec
            break

    return DOCTORS_DATABASE.get(specialty, DOCTORS_DATABASE["general"])

# ==============================================================
# 🌐 STREAMLIT APP
# ==============================================================
def main():
    st.set_page_config(
        page_title="Smart Health Assistant",
        page_icon="🏥",
        layout="wide",
        initial_sidebar_state="collapsed"
    )

    # Custom CSS with better visibility
    st.markdown("""
        <style>
        .stButton>button {
            width: 100%;
        }
        .doctor-card {
            padding: 1rem;
            border-radius: 10px;
            border: 2px solid #e0e0e0;
            margin-bottom: 1rem;
            background: #f9f9f9;
        }
        .doctor-name {
            font-size: 1.2rem;
            font-weight: bold;
            color: #1f77b4;
        }
        .doctor-info {
            margin: 0.3rem 0;
            color: #555;
        }
        .chat-section {
            background-color: #f0f8ff;
            padding: 20px;
            border-radius: 10px;
            margin-top: 20px;
            border: 2px solid #4CAF50;
        }
        .doctor-section {
            background-color: #fff3cd;
            padding: 20px;
            border-radius: 10px;
            margin-top: 20px;
            border: 2px solid #ff9800;
        }
        </style>
    """, unsafe_allow_html=True)

    # Initialize session states
    if "show_chatbot" not in st.session_state:
        st.session_state.show_chatbot = False
    if "show_doctors" not in st.session_state:
        st.session_state.show_doctors = False
    if "current_disease" not in st.session_state:
        st.session_state.current_disease = None
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = []
    if "prediction_done" not in st.session_state:
        st.session_state.prediction_done = False

    # Load resources
    disease_info, features, models, scaler, pca = load_resources()

    # ==============================================================
    # MAIN PAGE: DISEASE PREDICTOR
    # ==============================================================
    st.title("🏥 Smart Health Assistant")
    st.markdown("Enter your symptoms to get instant AI-powered disease predictions.")
    st.divider()

    if not all([disease_info, features, models, scaler, pca]):
        st.error("❌ Failed to load models.")
        st.stop()

    with st.expander("📋 View Sample Symptoms"):
        cols = st.columns(3)
        samples = features[:60]
        for idx, col in enumerate(cols):
            with col:
                st.markdown("\n\n".join([f"• {s}" for s in samples[idx*20:(idx+1)*20]]))

    st.markdown("### 🔍 Enter Your Symptoms")
    user_input = st.text_area(
        "Type symptoms separated by commas:",
        placeholder="Example: fever, cough, headache, fatigue",
        height=120
    )

    col1, col2, col3 = st.columns([1, 2, 1])
    with col2:
        predict_btn = st.button("🔬 Analyze Symptoms", type="primary", use_container_width=True)

    if predict_btn:
        if not user_input.strip():
            st.warning("⚠️ Please enter at least one symptom.")
        else:
            symptoms = [s.strip().lower() for s in user_input.split(",")]
            with st.spinner("🔄 Analyzing..."):
                results = predict_disease(symptoms, disease_info, features, models, scaler, pca)

            if not results:
                st.error("❌ No matches found.")
            else:
                st.session_state.prediction_done = True
                st.success(f"✅ Found {len(results)} predictions!")
                st.markdown("### 📊 Results")

                # Store the top disease for doctor recommendations
                if results:
                    st.session_state.current_disease = results[0][1]

                for rank, (model, pred, match, info) in enumerate(results, 1):
                    with st.expander(f"#{rank} - {pred.upper()} ({match}% match)", expanded=(rank==1)):
                        if match >= 70:
                            st.success(f"🎯 High Confidence: {match}%")
                        elif match >= 40:
                            st.warning(f"⚠️ Medium: {match}%")
                        else:
                            st.info(f"ℹ️ Low: {match}%")

                        st.progress(match / 100)

                        if info:
                            col1, col2 = st.columns(2)
                            with col1:
                                st.markdown("**🧩 Symptoms:**")
                                for s in info.get('symptoms', [])[:6]:
                                    st.markdown(f"• {s}")
                            with col2:
                                st.markdown("**📖 Description:**")
                                st.write(info.get('description', 'N/A'))

                            st.markdown("**💊 Treatment:**")
                            st.info(info.get('treatment', 'Consult a doctor.'))

    # ==============================================================
    # ACTION BUTTONS (Always visible after prediction)
    # ==============================================================
    if st.session_state.prediction_done:
        st.divider()
        st.markdown("### 🎯 What would you like to do next?")

        col1, col2, col3 = st.columns(3)

        with col1:
            if st.button("💬 Chat with AI Assistant", use_container_width=True, type="primary"):
                st.session_state.show_chatbot = not st.session_state.show_chatbot
                st.session_state.show_doctors = False
                st.rerun()

        with col2:
            if st.button("👨‍⚕️ Find Specialist Doctors", use_container_width=True, type="secondary"):
                st.session_state.show_doctors = not st.session_state.show_doctors
                st.session_state.show_chatbot = False
                st.rerun()

        with col3:
            if st.button("🔄 New Prediction", use_container_width=True):
                st.session_state.show_chatbot = False
                st.session_state.show_doctors = False
                st.session_state.prediction_done = False
                st.session_state.chat_history = []
                st.rerun()

    # ==============================================================
    # CHATBOT SECTION (Clearly visible when activated)
    # ==============================================================
    if st.session_state.show_chatbot:
        st.markdown('<div class="chat-section">', unsafe_allow_html=True)
        st.markdown("## 💬 Health Assistant Chatbot")
        st.markdown("**Ask me anything about your condition, treatment, or prevention!**")
        st.divider()

        # Display chat
        chat_container = st.container()
        with chat_container:
            if len(st.session_state.chat_history) == 0:
                st.info("👋 Hello! Ask me about your health concerns!")

            for msg in st.session_state.chat_history:
                if msg["role"] == "user":
                    with st.chat_message("user", avatar="🧑"):
                        st.markdown(msg["content"])
                else:
                    with st.chat_message("assistant", avatar="🤖"):
                        st.markdown(msg["content"])

        # Chat input
        user_question = st.chat_input("💬 Ask your question...")

        if user_question:
            st.session_state.chat_history.append({
                "role": "user",
                "content": user_question,
                "timestamp": datetime.now().strftime("%H:%M")
            })

            response = get_health_response(user_question, st.session_state.chat_history)

            st.session_state.chat_history.append({
                "role": "assistant",
                "content": response,
                "timestamp": datetime.now().strftime("%H:%M")
            })

            st.rerun()

        # Control buttons
        col1, col2 = st.columns(2)
        with col1:
            if st.button("🗑️ Clear Chat", use_container_width=True):
                st.session_state.chat_history = []
                st.rerun()
        with col2:
            if st.button("❌ Close Chatbot", use_container_width=True):
                st.session_state.show_chatbot = False
                st.rerun()

        st.markdown('</div>', unsafe_allow_html=True)

    # ==============================================================
    # DOCTORS SECTION (Clearly visible when activated)
    # ==============================================================
    if st.session_state.show_doctors:
        st.markdown('<div class="doctor-section">', unsafe_allow_html=True)
        st.markdown("## 👨‍⚕️ Recommended Specialist Doctors")

        if st.session_state.current_disease:
            st.info(f"📋 Showing specialists for: **{st.session_state.current_disease.upper()}**")
            doctors = get_specialists_for_disease(st.session_state.current_disease)
        else:
            st.info("📋 Showing general physicians")
            doctors = DOCTORS_DATABASE["general"]

        # Display doctors in cards
        cols = st.columns(2)
        for idx, doctor in enumerate(doctors):
            with cols[idx % 2]:
                st.markdown(f"""
                <div class="doctor-card">
                    <div class="doctor-name">👨‍⚕️ {doctor['name']}</div>
                    <div class="doctor-info">🏥 <b>Specialty:</b> {doctor['specialty']}</div>
                    <div class="doctor-info">📅 <b>Experience:</b> {doctor['experience']}</div>
                    <div class="doctor-info">🏢 <b>Hospital:</b> {doctor['hospital']}</div>
                    <div class="doctor-info">📞 <b>Phone:</b> {doctor['phone']}</div>
                    <div class="doctor-info">🕐 <b>Available:</b> {doctor['available']}</div>
                </div>
                """, unsafe_allow_html=True)

                if st.button(f"📞 Book Appointment with {doctor['name']}", key=f"book_{idx}", use_container_width=True):
                    st.success(f"✅ Appointment request sent to {doctor['name']}! They will contact you soon.")

        # Close button
        if st.button("❌ Close Doctor List", use_container_width=True):
            st.session_state.show_doctors = False
            st.rerun()

        st.markdown('</div>', unsafe_allow_html=True)

    # Footer
    st.divider()
    st.caption("⚠️ **Disclaimer:** This is an AI prediction tool. Always consult qualified healthcare professionals for accurate diagnosis and treatment.")

if __name__ == "__main__":
    main()

Writing app.py


In [ ]:
# Reinstall Groq
!pip uninstall groq -y
!pip install groq --upgrade

Found existing installation: groq 0.37.1
Uninstalling groq-0.37.1:
  Successfully uninstalled groq-0.37.1
  Using cached groq-0.37.1-py3-none-any.whl.metadata (16 kB)
Using cached groq-0.37.1-py3-none-any.whl (137 kB)


In [ ]:
# Add this to a new cell and run
import streamlit as st
st.cache_resource.clear()

In [ ]:


# Then run:
from pyngrok import ngrok
import subprocess, time, os

ngrok.set_auth_token("364Ot4apQ98O0fBzFGgwU1y4IwO_2XaDs42N8no5dgYNCxZt7")
try:
    ngrok.kill()
except:
    pass

process = subprocess.Popen([
    "streamlit", "run", "app.py",
    "--server.port", "8501",
    "--server.headless", "true"
])

time.sleep(10)
public_url = ngrok.connect(8501)

print(f"✅ APP IS LIVE!")
print(f"🌐 URL: {public_url.public_url}")

✅ APP IS LIVE!
🌐 URL: https://euphorically-unhumid-christy.ngrok-free.dev
